# FFT Implementation in Python
Use Python to implement FFT, just for learning the basic concepts

## Radix-2 FFT
Naive Implementation of FFT:
* Cooley-Turkey Framework
* Radix: 2
* padding 0 to deal with variable length


In [ ]:
# Recursive Version
import numpy as np
def fft_recursive(x):
    N = len(x)
    if N <= 1:
        return x

    even = fft_recursive(x[0::2])  # even_indices
    odd = fft_recursive(x[1::2])   # odd_indices

    # twiddle factor
    T = [np.exp(-2j * np.pi * k / N) * odd[k] for k in range(N // 2)]
    return [even[k] + T[k] for k in range(N // 2)] + \
           [even[k] - T[k] for k in range(N // 2)]


In [ ]:
# Iterative Version of FFT
def bit_reverse(n, bits):
    result = 0
    for i in range(bits):
        result <<= 1  # Shift result left by 1
        result |= (n & 1)  # Get the least significant bit of n and add it to result
        n >>= 1  # Shift n right by 1
    return result

def fft_iterative(x):
    N = len(x)
    # iterations needed
    logN = int(np.log2(N))

    # do bit-reverse
    indices = np.arange(N)
    bit_reversed_indices = [bit_reverse(i,logN) for i in range(N)]
    x = [x[i] for i in bit_reversed_indices]

    # FFT 
    for s in range(1, logN + 1):
        m = 1 << s 
        wm = np.exp(-2j * np.pi / m)  # 旋转因子
        for k in range(0, N, m):
            w = 1  # 初始化w
            for j in range(m // 2):
                t = w * x[k + j + m // 2]
                u = x[k + j]
                x[k + j] = u + t
                x[k + j + m // 2] = u - t
                w *= wm 

    return x

In [ ]:
# padding zero for calculation
def zero_padding(x):
    N = len(x)
    next_pow2 = 2 ** np.ceil(np.log2(N)).astype(int)
    padded_x = np.pad(x, (0, next_pow2 - N), 'constant')
    return padded_x;

In [ ]:
# Results
def formatRes(res):
  return [f"{val.real:.2f} + {val.imag:.2f}j" for val in res]
if __name__ == "__main__":
    print("Input: ")
    arrIn = np.arange(8)
    print(arrIn,"\n")

    print("Recursive Result: ")
    print("radix 2: ")
    print(formatRes(fft_recursive(arrIn)),"\n")

    print("Iterative Result: ")
    print(formatRes(fft_iterative(arrIn)),"\n")

    print("Random length FFT by padding 0: ")
    arrIn = np.arange(10)
    print("Input: ")
    print(arrIn,"\n")
    res = fft_iterative(zero_padding(arrIn))
    print(formatRes(res[:16]),"\n")




## Different Radix
Implement FFT using other radixes.

In [ ]:
import numpy as np
# radix 4 recursive FFT
def fft_recursive_r4(x):
    N = len(x)
    if N <= 1:
        return x
    if N % 4 != 0:
        raise ValueError("Length of input must be a power of 4.")
    
    # Split into four parts
    x0 = fft_recursive_r4(x[0::4])  # FFT of indices 0, 4, 8, ...
    x1 = fft_recursive_r4(x[1::4])  # FFT of indices 1, 5, 9, ...
    x2 = fft_recursive_r4(x[2::4])  # FFT of indices 2, 6, 10, ...
    x3 = fft_recursive_r4(x[3::4])  # FFT of indices 3, 7, 11, ...
    
    T = [np.exp(-2j * np.pi * k / N) for k in range(N // 4)]
    
    # Combine results
    result = [0] * N
    for k in range(N // 4):
        result[k] = x0[k] + T[k] * x1[k] + T[k]**2 * x2[k] + T[k]**3 * x3[k]
        result[k + N // 4] = x0[k] - 1j * T[k] * x1[k] - T[k]**2 * x2[k] + 1j * T[k]**3 * x3[k]
        result[k + 2 * N // 4] = x0[k] - T[k] * x1[k] + T[k]**2 * x2[k] - T[k]**3 * x3[k]
        result[k + 3 * N // 4] = x0[k] + 1j * T[k] * x1[k] - T[k]**2 * x2[k] - 1j * T[k]**3 * x3[k]
    
    return result

In [ ]:
# Results
def formatRes(res):
  return [f"{val.real:.2f} + {val.imag:.2f}j" for val in res]
if __name__ == "__main__":
    print("Input: ")
    arrIn = np.arange(16)
    print(arrIn)

    print("Recursive Result: ")
    print("radix 4: ")
    print(formatRes(fft_recursive_r4(arrIn)))
